# Data models and analyses techniques: Exercise sheet 1

#### *Your name:* [Please write your name here]

## Introduction

For each lecture there will be a collection of exercises presented as Jupyter notebooks. The exercises should be solved and documented as a mini-project that will form the basis for the examination. When solving the exercises it is therefore important that you 
 * document all relevant results and analyses that you have obtained/performed during the exercises.
 * try to relate your results to the theoretical background of the methods being applied. 

The documentation should be integrated (by adding new code/markdown cells) in the Jupyter notebooks containing the exercise sheets.

The mini-project should be delivered after the last seminar in this module (an exact date will be provided later) and should consist of:
 - a single pdf-file containing all the notebooks; the 'File' menu gives you the option of downloading a pdf-version of your notebook. The name of the file should be [your\_first\_name]_[your\_family\_name].pdf (e.g. Thomas\_Nielsen.pdf).
 - a copy of your notebooks with the solutions to the exercises. The notebooks should be named [your\_first name]\_[your\_family\_name]\_[exercise\_sheet\_number].ipynb (e.g., Thomas\_Nielsen\_1.ipnyb).
 
If you have any questions about the exercises, you are strongly encouraged to post you questions on the MS Teams channel or on the discussion forum on Moodle.

Last, but not least:
* Looking for an overview of the markdown language? The cheat sheet <a href="https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed">here</a> might help.
* For the Python specific components of the exercises, you should not need constructs beyond those that are already included in the notebooks on the course's web-page (still you should not feel constrained by these, so feel free to be adventurous). You may, however, need to consult the documentation for some of the methods supplied by `sklearn`.

# Data source

In this exercise we will play around with the a subset of the Boston housing data set, which is available at `http://lib.stat.cmu.edu/datasets/boston`. The data is unfortunately not available through sklearn, so instead we will have to download and massage it a bit.

In [ ]:
import pandas as pd
import requests
from io import StringIO
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

url = 'http://lib.stat.cmu.edu/datasets/boston'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the content of the response as a string and split into lines
    lines = response.text.splitlines()
    meta = "\n".join(lines[0:21])
    
    # Drop lines with meta data
    lines = lines[22:]
    
    # Column names
    col_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

    # Join every two lines (rows are broken over two lines)
    combined_lines = []
    for i in range(0, len(lines), 2):
        combined_line = lines[i].strip() + " " + lines[i + 1].strip()
        combined_lines.append(combined_line)

    # Combine all lines into a single string
    s = "\n".join(combined_lines)
    
    # Read into a DataFrame using StringIO
    df = pd.read_csv(StringIO(s), delim_whitespace=True, header=None, names=col_names)

    # Drop column B
    df = df.drop(columns=['B'])    

    data = {}
    data['DESCR'] = meta
    data['target'] = df['MEDV']
    data['data'] = df.drop(columns=['MEDV'])
    data['feature_names'] = data['data']

else:
    print(f"Failed to retrieve data: {response.status_code}")

In [ ]:
# The variables in the data set
data['data'].columns.to_list()

In [ ]:
# ... and the target variable with the median value of owner-occupied homes in $1000's
data['target']

Let's get a bit acquainted with the data

In [ ]:
print(data['DESCR'])

Again we treat it as a classification problem:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
c = np.array([1 if y > np.median(data['target']) else 0 for y in data['target']])
X_train, X_test, c_train, c_test = train_test_split(data['data'], c, random_state=0)

In [ ]:
# Create dataframe
data_df = pd.DataFrame(X_train, columns=data['data'].columns.to_list())
# Create scatter plot and color by class label.
%matplotlib inline
import matplotlib.pyplot as plt
_=pd.plotting.scatter_matrix(data_df, c=c_train, figsize=(15, 15), marker='o',
                           hist_kwds={'bins': 20}, s=60, alpha=.8)

# Exercise set 1

### Model learning
* Learn a decision tree using the training data and evaluate its performance on both the training data and the test data. Generate random training/test partitions or varying sizes and analyze how the accuracy results vary (consult the documentation for `train_test_split(.)`).

### Model analysis

* Display the decision tree learned using the training data.
* What are the most important features as determined by the learned tree and does, e.g., the choice of top node seem reasonable to you based on your knowledge of the data domain?
* How does the features deemed *most important* by the decision tree learner match the generated tree and your understanding of the domain?

### Model complexity
* Try controlling the complexity of the learned decision tree by adjusting the parameters max\_depth, min\_samples\_split, min\_samples\_leaf
* Investigate the effect when changing these parameters:
    - Visualize (some of) the trees
    - Evaluate the performance of the models on both the training data and the test data
* By designating a part of the data as *validation data*, try to automatically find good values for the parameters controlling the size of the tree. How does the obtained parameters match your manual investigation?
* Instead of using a validation set, try to find good parameter values using cross-validation. How does the results compare to those that you found above?

# Exercise set 2

In this exercise we shall expore and compare the k-nearest neighbor classifier and the naive Bayes model.

### K-nearest neighbor
* Classify the instances in the test set using default settings for the k-NN classifier.
* Try experimenting with different values for 'k'. How do the values affect the classification results? Feel free to try to automate this process using the methods explored in the previous exercise sheet. 
* Try to get some more insight into the data by generating descriptive statistics and by plotting the data. 
* Based on your understanding of the data and the properties of the k-NN classifier, does it seem reasonable to try to manipulate the data (e.g. through normalization) or work with different distance metrics in order to improve the performance of the classifier? How does such changes affect classification accuracy?

### Naive Bayes

* Classify the instances in the test set using the naive Bayes model.
* The naive Bayes classifier makes the strong assumption that the attributes are conditionally independent given the class variable. Can you identify any pairs of attributes that do not adhere to this assumption, and how does it affect the performance of the classifier if one of the attributes are removed?

### Model comparison

* Using the results obtained above, classify the instances in the test set using the k-NN classifier, the naive Bayes classifier, and decision trees.
* Based on your intuitive understanding of the classifiers, can you explain their relative performance? 